# **Install Library**

In [ ]:
!pip install IMDbPY

# **Import Libraries**

In [ ]:
from imdb import IMDb
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from textblob import TextBlob as tb
import random

# **Create empty list for extract the data from IMDB Website**

In [ ]:
ID= []
movie_name = []
movie_year = []
movie_genres = []
movie_directors = []
movie_writers = []
movie_rating = []
All_review= []
movie_budget= []
popularity=[]
user_review=[]

# **Create Function**

In [ ]:
# ------------------------------ Get Movie Information And Reviews -------------------------------------------
def get_movie_info(movie_id):
    obj_imdb = IMDb()
    movie = obj_imdb.get_movie(movie_id)
    # Retrieve movie information
    Id=movie.getID()
    title = movie.get('title')
    year = movie.get('year')
    genres = movie.get('genres')
    directors = movie.get('directors')
    writers = movie.get('writers')
    rating = movie.get('rating')
    # Append values to the lists
    ID.append(Id)
    movie_name.append(title)
    movie_year.append(year)
    movie_genres.append(','.join(genres))
    movie_directors.append(', '.join(director['name'] for director in directors))
    for writer in writers:
            writer_name = writer.get('name')
            if writer_name:
                movie_writers.append(writer_name)
    movie_rating.append(rating)
    # Retrieve Movie Reviews
    review=[]
    start_url = f'https://www.imdb.com/title/tt{movie_id}/reviews?ref_=tt_urv'
    link = f'https://www.imdb.com/title/tt{movie_id}/reviews/_ajax'
    params = {
    'ref_': 'undefined',
    'paginationKey': ''}
    with requests.Session() as s:
      s.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
      res = s.get(start_url)
      while True:
        soup = bs(res.text,"lxml")
        for item in soup.select(".review-container"):
            reviewer = item.select_one("a").get_text(strip=True)
            review.append(reviewer)
        try:
            pagination_key = soup.select_one(".load-more-data[data-key]").get("data-key")
        except AttributeError:
            break
        params['paginationKey'] = pagination_key
        res = s.get(link,params=params)
      All_review.append(review)
# ------------------- Sentiment Analysis For User Reviews about Movie -------------------------------------
def sentiment_analysis():
  for i in All_review:
    sentiment_scores = []
    for j in i:
      blob = tb(j)
      sentiment_score = blob.sentiment.polarity
      sentiment_scores.append(sentiment_score)
    aggregate_sentiment = sum(sentiment_scores) / len(sentiment_scores)
# Determine if the overall sentiment is positive or negative
    overall_sentiment = "Positive" if aggregate_sentiment >= 0.1 else "Negative"
    popularity.append(overall_sentiment)
# -------------------- Random any User Review ----------------------------------------------------------
def choose_random_review():
    for a in All_review:
      random_review = random.choice(a)
      user_review.append(random_review)



# **Function Call and append the movie details In above empty list**

In [ ]:
while True:
# Enter movie Id
    movie_id = input("Enter Movie ID (or 'q' to quit): ")
    if movie_id == 'q':
        break
    get_movie_info(movie_id)
# call sentiment_analysis function
sentiment_analysis()

Enter Movie ID (or 'q' to quit): 0068646
Enter Movie ID (or 'q' to quit): 1028532
Enter Movie ID (or 'q' to quit): 3138104
Enter Movie ID (or 'q' to quit): 0118799
Enter Movie ID (or 'q' to quit): 1853728
Enter Movie ID (or 'q' to quit): q


In [ ]:
# call random_review function for choose the any random user review
choose_random_review()

# **Create DataSet and Save the Dataset**

In [ ]:
dt=pd.DataFrame(list(zip(ID,movie_name,movie_year,movie_genres,movie_directors,movie_writers,movie_rating,user_review,popularity)),columns=['Movie_Id','Movie_Name','Movie_Year','Movie_Genres','Movie_Directors','Movie_Writers','Movie_Rating','User_Review','Popularity'])

In [ ]:
dt

,Movie_Id,Movie_Name,Movie_Year,Movie_Genres,Movie_Directors,Movie_Writers,Movie_Rating,User_Review,Popularity
0,0810913,Jack and Jill,2011,Comedy,Dennis Dugan,Ben Zook,3.3,Degrading!,Negative
1,0079470,Life of Brian,1979,Comedy,Terry Jones,Steve Koren,8.0,A stylistically eclectic comic masterpiece.,Positive
2,1392214,Prisoners,2013,"Crime,Drama,Mystery,Thriller",Denis Villeneuve,Adam Sandler,8.1,Denis Villeneuve is a cruel man without a sing...,Positive
3,7286456,Joker,2019,"Crime,Drama,Thriller",Todd Phillips,Allen Covert,8.4,Amazing,Positive
4,0133093,The Matrix,1999,"Action,Sci-Fi","Lana Wachowski, Lilly Wachowski",Robert Smigel,8.7,Fantastic!,Positive
5,0086456,The Little Girl Who Conquered Time,1983,"Romance,Sci-Fi",Nobuhiko Ôbayashi,Robert Smigel,6.6,Not Bad Teen Idol Flick,Positive
6,0036868,The Best Years of Our Lives,1946,"Drama,Romance,War",William Wyler,Graham Chapman,8.1,A Sensitive Portrait Of Re-Acclimation,Positive
7,0103639,Aladdin,1992,"Animation,Adventure,Comedy,Family,Fantasy,Musi...","Ron Clements, John Musker",John Cleese,8.0,Solid Gold Disney Classic!!!,Positive
8,4667094,Fifty Shades of Black,2016,Comedy,Michael Tiddes,Terry Gilliam,3.5,Great Parody Movie,Negative
9,1883367,The Human Centipede III (Final Sequence),2015,"Comedy,Horror",Tom Six,Eric Idle,2.8,"Sick, sick fun!!!",Negative


In [ ]:
dt.to_csv('movie_dataset.csv')